In [1]:
import pyhector
from pyhector import rcp26, rcp45, rcp60, rcp85

In [ ]:
# Input emissions rcp 2.6
rcp26.head(5)

In [ ]:
# Run hector and show default outputs
output = pyhector.run(rcp26)
output.head(5)

In [ ]:
# Display other outputs
output = pyhector.run(rcp26, outputs=['temperature.Tgav', 'simpleNbox.Ca', 'forcing.Ftot',
                                      'forcing.FCO2', 'ocean.Temp_HL'])
output.head(5)

In [ ]:
# Calculate mean forcing 1850-1950
output['forcing.Ftot'].loc[1850:1950].mean()

In [ ]:
# Plot rcp temperatures
import matplotlib.pyplot as plt

for rcp in [rcp26, rcp45, rcp60, rcp85]:
    output = pyhector.run(rcp, {"core": {"endDate": 2100}})
    temp = output["temperature.Tgav"]
    # Adjust to 1850 - 1900 reference period
    temp = temp.loc[1850:] - temp.loc[1850:1900].mean()
    temp.plot(label=rcp.name.split("_")[0])
plt.title("Global mean temperature")
plt.ylabel("Degrees C over pre-industrial (1850-1900 mean)")
plt.legend(loc="best")
plt.show()

In [ ]:
# Read in co2 file to match
import pandas as pd
from numpy import mean
from dplython import (DplyFrame, X, mutate)
co2_file_to_match = "annual_avg_co2_GFDL-ESM2M_rcp45_r1i1p1.csv"
esm_co2_data = pd.read_csv(co2_file_to_match).rename(columns={"value.1...":"co2_value"}).set_index('year')
esm_co2 = esm_co2_data.co2_value * 1000000
esm_co2.head(5)

In [ ]:
# Compare hector and esm
CONCENTRATION_CO2 = "simpleNbox.Ca"
hector_co2 = pyhector.run(pyhector.rcp45)[CONCENTRATION_CO2].loc[esm_co2.index]
DplyFrame({"hector": hector_co2, "esm": esm_co2}).head()

In [ ]:
# Function to calculate difference
def difference_quantifier(esm_series, hector_run_series):
    calculate_df = DplyFrame({"hector": hector_run_series, "esm": esm_series})
    calculate_df = calculate_df >> mutate(percentdiff=(X.hector - X.esm) / X.esm)
    return mean(abs(calculate_df.percentdiff))

difference_quantifier(esm_co2,hector_co2)

In [ ]:
# Function to run hector with selected parameters and calculate difference from esm
def hector_runner(params, comp_data, var):
    hector_output = pyhector.run(pyhector.rcp45, {"temperature": {"S": params[0]},
                                                 "simpleNbox":{"beta":params[1]},
                                                "simpleNbox":{"q10_rh":params[2]}})
    hector_co2 = hector_output[var].loc[comp_data.index]
    return difference_quantifier(comp_data, hector_co2)

hector_runner([4,1,1], esm_co2, CONCENTRATION_CO2)

In [ ]:
import scipy.optimize

In [ ]:
optim = scipy.optimize.fmin_powell(hector_runner, x0 = [5,2,2], args = (esm_co2,CONCENTRATION_CO2))
optim

In [ ]:
# Compare optimized hector to default hector
hector_optim =  pyhector.run(pyhector.rcp45, {"temperature": {"S": 3.74202374},
                                                 "simpleNbox":{"beta":9.77973754},
                                                "simpleNbox":{"q10_rh":2.00205191}})

optimized = hector_optim[CONCENTRATION_CO2].loc[2010:2100]

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 16, 9

hector_co2.plot(label="Default Hector")
optimized.plot(label = "Optimized Hector")
esm_co2.plot(label = "GFDL-ESM2M")
plt.title("CO2 Concentration - rcp45")
plt.ylabel("PPM")
plt.legend(loc="best")
plt.show()